In [1]:
# Data preprocessing
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from glob import glob
from PIL import Image
import os
import random
import cv2
#Model
import keras
from keras.models import Sequential, Model,load_model
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPooling2D,MaxPool2D,AveragePooling2D,GlobalMaxPooling2D
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras import regularizers
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator,array_to_img
from keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint
from keras.metrics import PrecisionAtRecall,Recall 
#Model Analysis
from sklearn.metrics import confusion_matrix

In [46]:
path='../input/covid19data'

diagnosis_code_dict={
    'COVID':0,
    'Normal':1,
    'Viral Pneumonia':2
}

diagnosis_label_dict={
    'COVID':'Covid-19',
    'Normal':'Halthy Lungs',
    'Viral Pneumonia':'Viral Pneumonia'
}

image_path_dict={os.path.splitext(os.path.basename(x))[0]:x for x in glob(os.path.join(path,'*','*.png'))}

covidData=pd.DataFrame.from_dict(image_path_dict,orient='index').reset_index()

covidData.columns=['image_id','path']
classes=covidData.image_id.str.split('-').str[0]
covidData['diagnosis']=classes
covidData['target']=covidData['diagnosis'].map(diagnosis_code_dict.get)
covidData['Class']=covidData['diagnosis'].map(diagnosis_label_dict.get)

In [47]:
samples,features=covidData.shape
duplicate=covidData.duplicated().sum()
null_val=covidData.isnull().sum()

print(samples)
print(features)
print(duplicate)
print(null_val)

In [48]:
covidData

In [49]:
from decimal import Decimal
plt.figure(figsize=(10,6))
ax=sns.countplot(data=covidData,x='Class',order=covidData['Class'].value_counts().index)
for p in ax.patches:
    ax.annotate('{}'.format(Decimal(str(p.get_height()))), (p.get_x(), p.get_height()),ha='center', va='bottom', color='black', xytext=(75,5),rotation = 'horizontal',
    textcoords='offset points')
plt.ylim(0,11500)
plt.title('Number of Samples per Class',fontsize=16,weight='bold')
ax.set_xlabel('Sample Type - Diagnosis',weight='bold')
ax.set_ylabel('Count',weight='bold')
plt.savefig('bar chart distribution.png')

In [58]:
#Samples per class
plt.figure(figsize=(20,8))
sns.set(style="ticks", font_scale = 1)
ax = sns.countplot(data = covidData,x='Class',order = covidData['Class'].value_counts().index)
sns.despine(top=True, right=True, left=True, bottom=False)
plt.xticks(rotation=0,fontsize = 12)
ax.set_xlabel('Sample Type - Diagnosis',fontsize = 14,weight = 'bold')
ax.set(yticklabels=[])
ax.axes.get_yaxis().set_visible(False) 
plt.title('Number of Samples per Class', fontsize = 16,weight = 'bold')
#Plot numbers
for p in ax.patches:
    ax.annotate("%.1f%%" % (100*float(p.get_height()/samples)), (p.get_x() + p.get_width() / 2., abs(p.get_height())),
    ha='center', va='bottom', color='black', xytext=(0, 10),rotation = 'horizontal',
    textcoords='offset points')

In [51]:
healthy_lungs=(10192/15153)*100
covid_19=(3616/15153)*100
viral_pneumonia=(1345/15153)*100

print(healthy_lungs)
print(covid_19)
print(viral_pneumonia)

In [52]:
fig,ax=plt.subplots()
labels = ['Healthy Lungs', 
         'Covid-19',
         'Viral Pneumonia']
percentages = [67.26,23.86,8.87]
ax.pie(percentages, labels=labels,  autopct='%1.1f%%',shadow=False, startangle=0,   pctdistance=1.2,labeldistance=1.4)
ax.axis('equal')
#ax.set_title("Percentage of Classes")
ax.legend(frameon=False, bbox_to_anchor=(1.5,0.8))
plt.savefig('percentage-pie chart.png')

In [53]:
plt.figure()
pic_id = random.randrange(0, samples)
picture = covidData['path'][pic_id]
image = cv2.imread(picture)
plt.imshow(image)
plt.axis('off');
plt.show()   
plt.savefig("sample.jpg")

In [54]:
print('Shape of the image : {}'.format(image.shape))
print('Image Hight {}'.format(image.shape[0]))
print('Image Width {}'.format(image.shape[1]))
print('Dimension of Image {}'.format(image.ndim))
print('Image size {}'.format(image.size))
print('Image Data Type {}'.format(image.dtype))
print('Maximum RGB value in this image {}'.format(image.max()))
print('Minimum RGB value in this image {}'.format(image.min()))

In [55]:
plt.title('B channel',fontsize = 14,weight = 'bold')
plt.imshow(image[ : , : , 0])
plt.axis('off');
plt.show()
plt.savefig("Blue channel of sample.jpg")

In [56]:
covidData['image'] = covidData['path'].map(lambda x: np.asarray(Image.open(x).resize((100,100))))

In [57]:
#Image Sampling
n_samples = 3

fig, m_axs = plt.subplots(3, n_samples, figsize = (4*n_samples, 3*4))

for n_axs, (type_name, type_rows) in zip(m_axs,covidData.sort_values(['diagnosis']).groupby('diagnosis')):
    n_axs[1].set_title(type_name,fontsize = 14,weight = 'bold')
    for c_ax, (_, c_row) in zip(n_axs, type_rows.sample(n_samples, random_state=1234).iterrows()):       
        picture = c_row['path']
        image = cv2.imread(picture)
        c_ax.imshow(image)
        c_ax.axis('off')
plt.savefig('sample images of classes.png')

In [60]:
mean_val = []
std_dev_val = []
max_val = []
min_val = []

for i in range(0,samples):
    mean_val.append(covidData['image'][i].mean())
    std_dev_val.append(np.std(covidData['image'][i]))
    max_val.append(covidData['image'][i].max())
    min_val.append(covidData['image'][i].min())

imageEDA = covidData.loc[:,['image', 'Class','path']]
imageEDA['mean'] = mean_val
imageEDA['stedev'] = std_dev_val
imageEDA['max'] = max_val
imageEDA['min'] = min_val

subt_mean_samples = imageEDA['mean'].mean() - imageEDA['mean']
imageEDA['subt_mean'] = subt_mean_samples

In [61]:
ax = sns.displot(data = imageEDA, x = 'mean', kind="kde");
plt.title('Images Colour Mean Value Distribution', fontsize = 16,weight = 'bold');
ax = sns.displot(data = imageEDA, x = 'mean', kind="kde", hue = 'Class');
plt.title('Images Colour Mean Value Distribution by Class', fontsize = 16,weight = 'bold');
ax = sns.displot(data = imageEDA, x = 'max', kind="kde", hue = 'Class');
plt.title('Images Colour Max Value Distribution by Class', fontsize = 16,weight = 'bold');
ax = sns.displot(data = imageEDA, x = 'min', kind="kde", hue = 'Class');
plt.title('Images Colour Min Value Distribution by Class', fontsize = 16,weight = 'bold');

In [62]:
plt.figure(figsize=(20,8))
sns.set(style="ticks", font_scale = 1)
ax = sns.scatterplot(data=imageEDA, x="mean", y=imageEDA['stedev'], hue = 'Class',alpha=0.8);
sns.despine(top=True, right=True, left=False, bottom=False)
plt.xticks(rotation=0,fontsize = 12)
ax.set_xlabel('Image Channel Colour Mean',fontsize = 14,weight = 'bold')
ax.set_ylabel('Image Channel Colour Standard Deviation',fontsize = 14,weight = 'bold')
plt.title('Mean and Standard Deviation of Image Samples', fontsize = 16,weight = 'bold');
plt.savefig('Mean and Standard Deviation of Image Samples.jpg')

In [63]:
plt.figure(figsize=(20,8));
g = sns.FacetGrid(imageEDA, col="Class",height=5);
g.map_dataframe(sns.scatterplot, x='mean', y='stedev',color="gray");
g.set_titles(col_template="{col_name}", row_template="{row_name}", size = 16)
g.fig.subplots_adjust(top=.7)
g.fig.suptitle('Mean and Standard Deviation of Image Samples',fontsize=16, weight = 'bold')
axes = g.axes.flatten()
axes[0].set_ylabel('Standard Deviation');
for ax in axes:
    ax.set_xlabel('Mean')
g.fig.tight_layout()
plt.savefig('Mean and Standard Deviation of Image Samples different classes.jpg')

# CNN MODEL

In [66]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten, AveragePooling2D
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
import numpy as np
import pandas as pd
import shutil
import time
import cv2 as cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
sdir=r'../input/covid19data'
classlist=os.listdir(sdir)    
filepaths=[]
labels=[]    
for klass in classlist:
    classpath=os.path.join(sdir,klass)        
    flist=os.listdir(classpath)        
    for f in flist:
        fpath=os.path.join(classpath,f)        
        filepaths.append(fpath)
        labels.append(klass) 
    Fseries=pd.Series(filepaths, name='filepaths')
    Lseries=pd.Series(labels, name='labels')

In [4]:
df=pd.concat([Fseries, Lseries], axis=1) 

In [5]:
df.shape

In [6]:
height=150
width=150
channels=3
batch_size=100
img_shape=(height, width, channels)
img_size=(height, width)

train_df, dummy_df=train_test_split(df, train_size=0.80, shuffle=True, random_state=123)
test_df, valid_df=train_test_split(dummy_df, train_size=0.85, shuffle=True, random_state=123)


#Define the parameters to create the training and validation set Images and Data Augmentation parameters
trgen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2)

#**No Augmentation on the Test set Images**
tvgen = ImageDataGenerator(rescale=1./255)


train_gen=trgen.flow_from_dataframe( train_df, 
                                    x_col='filepaths', 
                                    y_col='labels', 
                                    target_size=img_size, 
                                    class_mode='categorical',
                                    color_mode='rgb', 
                                    shuffle=False, 
                                    batch_size=batch_size)

test_gen=tvgen.flow_from_dataframe( test_df, 
                                   x_col='filepaths', 
                                   y_col='labels', 
                                   target_size=img_size, 
                                   class_mode='categorical',
                                    color_mode='rgb', 
                                   shuffle=False, 
                                   batch_size=batch_size)

valid_gen=tvgen.flow_from_dataframe( valid_df, 
                                    x_col='filepaths', 
                                    y_col='labels', 
                                    target_size=img_size, 
                                    class_mode='categorical',
                                    color_mode='rgb', 
                                    shuffle=False, 
                                    batch_size=batch_size)

In [7]:
classes=list(train_gen.class_indices.keys())
class_count=len(classes)

In [2]:

    model=Sequential([
    Conv2D(64,(3,3),activation='relu',input_shape=(150,150,3)),
    BatchNormalization(),

    Conv2D(64,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
        
    Conv2D(64,(3,3),activation='relu'),
    BatchNormalization(),
        
    Conv2D(64,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(128,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(128,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(256,(3,3),activation='relu'),
    BatchNormalization(),

    Conv2D(256,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(256,activation='relu'),
    BatchNormalization(),
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    BatchNormalization(),
    Dense(3,activation='softmax')
])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 148, 148, 64)      256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 146, 146, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        3

In [9]:
from tensorflow.keras.callbacks import Callback,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
lrd = ReduceLROnPlateau(monitor='val_loss', patience=10, verbose=0, factor=0.5, min_lr=0.00001)
es = EarlyStopping(patience=100,monitor='val_loss', mode = 'min',verbose=0)

In [10]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [11]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]

In [12]:
model.compile(Adam(lr=.001), loss='categorical_crossentropy', metrics=METRICS)

In [13]:
history=model.fit(train_gen,validation_data=valid_gen,epochs=100,callbacks=[lrd,es])

In [64]:
result=model.evaluate(test_gen,verbose=1)

In [74]:
test_labels=test_gen.classes 
predictions=model.predict(test_gen)
y_pred = np.argmax(predictions, axis=-1)
print(classification_report(test_labels, y_pred))
print(confusion_matrix(test_labels, y_pred))

In [75]:
matrix = confusion_matrix(test_labels, y_pred)
print(matrix)
plt.figure(figsize=(8,6))
sns.heatmap(matrix, annot=True)
plt.savefig("confusion matrix")

In [24]:
def Train_Val_Plot(acc,val_acc,loss,val_loss,auc,val_auc,precision,val_precision,recall,val_recall,f1,val_f1):
    
    fig, (ax1, ax2,ax3,ax4,ax5,ax6) = plt.subplots(1,6, figsize= (30,5))
    fig.suptitle(" MODEL'S METRICS VISUALIZATION ")

    ax1.plot(range(1, len(acc) + 1), acc)
    ax1.plot(range(1, len(val_acc) + 1), val_acc)
    ax1.set_title('History of Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.legend(['training', 'validation'])


    ax2.plot(range(1, len(loss) + 1), loss)
    ax2.plot(range(1, len(val_loss) + 1), val_loss)
    ax2.set_title('History of Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend(['training', 'validation'])
    
    ax3.plot(range(1, len(auc) + 1), auc)
    ax3.plot(range(1, len(val_auc) + 1), val_auc)
    ax3.set_title('History of AUC')
    ax3.set_xlabel('Epochs')
    ax3.set_ylabel('AUC')
    ax3.legend(['training', 'validation'])
    
    ax4.plot(range(1, len(precision) + 1), precision)
    ax4.plot(range(1, len(val_precision) + 1), val_precision)
    ax4.set_title('History of Precision')
    ax4.set_xlabel('Epochs')
    ax4.set_ylabel('Precision')
    ax4.legend(['training', 'validation'])
    
    ax5.plot(range(1, len(f1) + 1), recall)
    ax5.plot(range(1, len(val_f1) + 1), val_recall)
    ax5.set_title('History of Recall')
    ax5.set_xlabel('Epochs')
    ax5.set_ylabel('Recall')
    ax5.legend(['training', 'validation'])
    
    ax6.plot(range(1, len(f1) + 1), f1)
    ax6.plot(range(1, len(val_f1) + 1), val_f1)
    ax6.set_title('History of F1-score')
    ax6.set_xlabel('Epochs')
    ax6.set_ylabel('F1 score')
    ax6.legend(['training', 'validation'])


    plt.show()
    

Train_Val_Plot(history.history['accuracy'],history.history['val_accuracy'],
               history.history['loss'],history.history['val_loss'],
               history.history['auc'],history.history['val_auc'],
               history.history['precision'],history.history['val_precision'],
               history.history['recall'],history.history['val_recall'],
               history.history['f1_score'],history.history['val_f1_score']
              )

In [71]:
def Train_Val_Plot1(acc,val_acc,loss,val_loss,auc,val_auc):
    
    fig, (ax1, ax2,ax3) = plt.subplots(1,3, figsize= (25,5))
    fig.suptitle(" MODEL'S METRICS VISUALIZATION ")

    ax1.plot(range(1, len(acc) + 1), acc)
    ax1.plot(range(1, len(val_acc) + 1), val_acc)
    ax1.set_title('Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.legend(['training', 'validation'])


    ax2.plot(range(1, len(loss) + 1), loss)
    ax2.plot(range(1, len(val_loss) + 1), val_loss)
    ax2.set_title('Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend(['training', 'validation'])
    
    ax3.plot(range(1, len(auc) + 1), auc)
    ax3.plot(range(1, len(val_auc) + 1), val_auc)
    ax3.set_title('AUC')
    ax3.set_xlabel('Epochs')
    ax3.set_ylabel('AUC')
    ax3.legend(['training', 'validation'])
    

    plt.show()
    

Train_Val_Plot1(history.history['accuracy'],history.history['val_accuracy'],
               history.history['loss'],history.history['val_loss'],
               history.history['auc'],history.history['val_auc']
              )

In [72]:
def Train_Val_Plot2(precision,val_precision,recall,val_recall,f1,val_f1):
    
    fig, (ax4,ax5,ax6) = plt.subplots(1,3, figsize= (25,5))
    fig.suptitle(" MODEL'S METRICS VISUALIZATION ")

   
    ax4.plot(range(1, len(precision) + 1), precision)
    ax4.plot(range(1, len(val_precision) + 1), val_precision)
    ax4.set_title('Precision')
    ax4.set_xlabel('Epochs')
    ax4.set_ylabel('Precision')
    ax4.legend(['training', 'validation'])
    
    ax5.plot(range(1, len(f1) + 1), recall)
    ax5.plot(range(1, len(val_f1) + 1), val_recall)
    ax5.set_title('Recall')
    ax5.set_xlabel('Epochs')
    ax5.set_ylabel('Recall')
    ax5.legend(['training', 'validation'])
    
    ax6.plot(range(1, len(f1) + 1), f1)
    ax6.plot(range(1, len(val_f1) + 1), val_f1)
    ax6.set_title('F1-score')
    ax6.set_xlabel('Epochs')
    ax6.set_ylabel('F1 score')
    ax6.legend(['training', 'validation'])

    plt.show()
    

Train_Val_Plot2(history.history['precision'],history.history['val_precision'],
               history.history['recall'],history.history['val_recall'],
               history.history['f1_score'],history.history['val_f1_score']
              )

In [31]:
#plotting training values
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.figure(figsize=(15,5))
plt.plot(epochs, acc, color='blue', label='Training Accuracy')
plt.plot(epochs, val_acc, color='gray', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

In [32]:
#plot loss
plt.figure(figsize=(15,5))
plt.plot(epochs, loss, color='blue', label='Training Loss')
plt.plot(epochs, val_loss, color='gray', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [35]:
#plot auc
auc = history.history['auc']
val_auc = history.history['val_auc']
plt.figure(figsize=(15,5))
plt.plot(epochs, auc, color='blue', label='Training AUC')
plt.plot(epochs, val_auc, color='gray', label='Validation AUC')
plt.title('Training and Validation AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.show()

In [38]:
#plot precision
precision = history.history['precision']
val_precision = history.history['val_precision']
plt.figure(figsize=(15,5))
plt.plot(epochs, precision, color='blue', label='Training Precision')
plt.plot(epochs, val_precision, color='gray', label='Validation Precision')
plt.title('Training and Validation Precison')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.show()

In [39]:
#plot recall
recall = history.history['recall']
val_recall = history.history['val_recall']
plt.figure(figsize=(15,5))
plt.plot(epochs, recall, color='blue', label='Training Recall')
plt.plot(epochs, val_recall, color='gray', label='Validation Recall')
plt.title('Training and Validation Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.show()

In [41]:
#plot f1 score
f1 = history.history['f1_score']
val_f1 = history.history['val_f1_score']
plt.figure(figsize=(15,5))
plt.plot(epochs, f1, color='blue', label='Training F1 Score')
plt.plot(epochs, val_f1, color='gray', label='Validation F1 Score')
plt.title('Training and Validation F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.show()

In [59]:
model.save('covid-19 model.h5')

In [83]:
from decimal import Decimal
df1 = pd.DataFrame({
    'Class': ['Training set','Validation set','Testing set'],
    'Accuracy':[0.9896,0.9546,0.9398],
    'Precision': [0.9829,0.9187,0.9099],
    'Recall': [0.9828,0.9187,0.9095],
    'F-1 Score':[0.9928,0.9202,0.9096],
    'AUC':[0.9991,0.9788,0.9844],
})

print(df1)
df1.head(10)

sns.set_style("darkgrid")
ax=df1.plot(x="Class",y=["Accuracy","Precision","Recall","F-1 Score","AUC"],kind="bar",width=0.8,figsize=(10,5))
for p in ax.patches:
    ax.annotate('{:.2f}'.format(Decimal(str(p.get_height()))), (p.get_x(), p.get_height()))
plt.xlabel("CLASSES",fontweight='bold')
plt.ylabel("MEASURE",fontweight='bold')
plt.xticks(rotation=0)
plt.ylim(0.8,1)
plt.legend(bbox_to_anchor=(1, 1),loc='upper left')
plt.show()
# plt.savefig("performance comparison of the classes.png")

In [86]:
from decimal import Decimal
df1 = pd.DataFrame({
    'Class': ['Normal','Viral Pneumonia','Covid'],
    'Precision': [0.76,0.98,0.93],
    'Recall': [0.99,0.88,0.94],
    'F-1 Score':[0.85,0.93,0.93]
})

print(df1)
df1.head(10)

sns.set_style("darkgrid")
ax=df1.plot(x="Class",y=["Precision","Recall","F-1 Score"],kind="bar",width=0.8,figsize=(6,4))
for p in ax.patches:
    ax.annotate('{:.2f}'.format(Decimal(str(p.get_height()))), (p.get_x(), p.get_height()))
plt.xlabel("CLASSES",fontweight='bold')
plt.ylabel("MEASURE",fontweight='bold')
plt.xticks(rotation=0)
plt.ylim(0.7,1)
plt.legend(bbox_to_anchor=(1, 1),loc='upper left')
plt.show()
# plt.savefig("performance comparison of the classes.png")